# Analysis on Identifying Cats with AWS Rekognition API

## OS environment set up

In [ ]:
!pip install boto3
!pip install pyfs
!mkdir temp

## Useful function used

This is a helper function to get the API's Cat's confidence.

In [ ]:
import boto3
from botocore.errorfactory import ClientError
import PIL
import numpy as np


def detect_labels(photo, bucket, local, p = False):
    ## link to the AWS ML service
    client=boto3.client('rekognition')
    response = None
    ## provide option to read image from both local and s3 buckets
    if local:
        with open(photo, 'rb') as image_file:
            print("img read")
            image = image_file.read()
            print(type(image))
            response = client.detect_labels(Image={'Bytes': image}, MaxLabels=10)
    else:
        response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
            MaxLabels=10)
    
    # optional logging
    if p:
        print('Detected cat labels for ' + photo) 
        print()
    found_cat = False
    for label in response['Labels']:
        if label['Name'] == 'Cat':
            found_cat = True
            for instance in label['Instances']:
                ## optional logging
                if p:
                    print ("  Bounding box")
                    print ("    Top: " + str(instance['BoundingBox']['Top']))
                    print ("    Left: " + str(instance['BoundingBox']['Left']))
                    print ("    Width: " +  str(instance['BoundingBox']['Width']))
                    print ("    Height: " +  str(instance['BoundingBox']['Height']))
                    print ("  Confidence: " + str(instance['Confidence']))
                    print()
            return found_cat, label
    return found_cat, 0

### A Sample Run on the above function

Display the image:

In [ ]:
import s3fs
fs = s3fs.S3FileSystem()
file = fs.open('s3://qtm350project/1181667592680_.pic.jpg')
image = PIL.Image.open(file)
display(image)

Show sample result on the image

In [ ]:
res = detect_labels('1181667592680_.pic.jpg', 'qtm350project', False, True)
res

Do some manipulation to the image and run the ML API again

In [ ]:
image_inv = PIL.ImageOps.invert(image)
display(image_inv)
image_inv.save('./img/sample_img/inv_1181667592680_.pic.jpg')

In [ ]:
res = detect_labels('./img/sample_img/inv_1181667592680_.pic.jpg', 'qtm350project', True)
res

## Analysis

### Get data

In the baseline run, all sample image taken by the group member would run through the AWS ML API
<!-- more explanation -->

Get the names of all images

In [ ]:
s3_resource = boto3.resource('s3')
my_bucket = s3_resource.Bucket('qtm350project')
summaries = my_bucket.objects.all()
image_names = [image.key for image  in summaries]
image_names

Fetch all images from s3 bucket

In [ ]:
for name in image_names:
    try:
        s3_resource.Bucket('qtm350project').download_file(name, f'/temp/original/{name}')
    except:
        print(f'Unable to get {name}')

### data manipulation

Some data manipulation would be done on the original image to investigate the effect of how different operation would impact AWS ML performance.  
- Rotation
    - rotation of 60, 90, and 180 degrees would be discussed
- Cut and Shuffle
    - cut the image onto four pieces and rearrange it in different order
- Obsure 这边可以改下称呼
    - using maxpool and minpool to make the image more obscure and harder to read

#### Rotation

#### Cut and Shuffle

In [ ]:
import random

def crop(img, order):
    raw = np.array(img)
    h = raw.shape[0]
    w = raw.shape[1]
    divide1 = int(h/2)
    divide2 = int(w/2)
    scattered = []
    scattered.append(raw[0:divide1, 0:divide2])
    scattered.append(raw[divide1:2*divide1, 0:divide2])
    scattered.append(raw[0:divide1, divide2:2*divide2])
    scattered.append(raw[divide1:2*divide1, divide2:2*divide2])

    raw[0:divide1, 0:divide2] = scattered[order[0]]
    raw[divide1:2*divide1, 0:divide2] = scattered[order[1]]
    raw[0:divide1, divide2:2*divide2] = scattered[order[2]]
    raw[divide1:2*divide1, divide2:2*divide2] = scattered[order[3]] 
    return raw

def get_order():
    flag = True
    order = None
    while flag:
        order = random.sample([1,2,3,4],4)
        if [1,2,3,4] != order:
            flag = False
    return order

fs = os.listdir("temp/")
for item in fs:
    if "MAX" not in item:
        try:
            Image.fromarray(crop(Image.open("temp/" + item))).save("temp/"  + "CROPPED" + item)
        except:
            print("Problem:" + item)

#### Obsure 待改

In [ ]:
def squeeze(img, f, isMax): #f is the length of the filter box, f=3 means 3x3 box
    raw = np.array(img)
    h = raw.shape[0]
    w = raw.shape[1]
    after = raw
    for i in range(0, h-f):
        for j in range(0, w-f):
            if isMax == True: v = np.max(raw[i:i+f, j:j+f])
            else: v = np.mean(raw[i:i+f, j:j+f])
            after[i,j] = v
    return after[0:h-f,0:w-f]

### Run ML API
Below is a helper funtion to run ML API on all files in a given directory

In [ ]:
import os

def predict(dir_path):
    rsts = {}
    ls = os.listdir(dir_path)
    for l in ls:
        if l[-3:].lower() == "png" or l[-3:].lower() == "jpg":
            try:
                res = detect_labels(dir_path + l, None, True)
                rsts[l] = res
            except:
                print(f'Error in running ML API on {l}')
    return rsts

#### Baseline Run
In the baseline run, all sample image taken by the group member would run through the AWS ML API
<!-- more explanation -->

In [ ]:
conf_orig = predict("/temp/original")

### Analysis on the effect of rotation

### Analysis on the effect of clarity